<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/Valorant_Bot_LLM_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# DO NOT SET THIS TO EXTRA INDEX URL, YOU WILL MESS STUFF UP BADLY WIKTH PERFORAMNCE




!pip install diskcache flask_ngrok2 pyngrok
!pip install -U llama-cpp-python --index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
from llama_cpp import Llama


Looking in indexes: https://abetlen.github.io/llama-cpp-python/whl/cu122
  Using cached https://github.com/abetlen/llama-cpp-python/releases/download/v0.2.87-cu122/llama_cpp_python-0.2.87-cp310-cp310-linux_x86_64.whl (394.5 MB)


In [34]:




# a little code is taken from a previous project
class llamaLLM:
    def __init__(self, systemprompt, model="bartowski/Meta-Llama-3-8B-Instruct-GGUF", filename="Meta-Llama-3-8B-Instruct-Q6_K.gguf"):


        self.messages = [
            # Sample data fed because I couldn't think of a way to get it to say the right shit
          {"role": "system", "content": systemprompt},
          {"role": "user", "content": "(Team) teammate: clove hit 70"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Clove -70"},
          {"role": "user", "content": "(Team) teammate: brim -140"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Brimstone -140"},
          {"role": "user", "content": "(Team) teammate: sage 80"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Sage -80"}
        ]

        self.llm = Llama.from_pretrained(
            repo_id=model,
            filename=filename,
            verbose=False,
            n_gpu_layers=-1,
            n_ctx=4096,
        )
    def addToConversation(self, content, role):
        self.messages.append(
              {
                  "role": role,
                  "content": content
              }
          )
    def generateConversationResponse(self, message, person="user"):
        self.addToConversation(message, person) # it can also be system
        response = self.llm.create_chat_completion(self.messages)["choices"][0]['message']['content'] # forgive me
        self.addToConversation(response, "assistant")
        return response # no genuinely what the fuck is this



#while True:
#    print(llm.generateConversationResponse(input(">> ")))


In [37]:
llm = llamaLLM("""
        You are a gamer playing Valorant, and are talking in the in-game chat.

        Your teammates may type out how much damage they have done to certain enemies, for example: '(Party) iopi: jett 40' or '(Party) KingAji15: sage -120'. This means that they have done 40 damage to Jett and 120 damage to Sage.
        If this sent, you are required respond in the format of '[HEALTHINDICATOR] {Agent} {Damange}', where agent, (Any valorant agent) and damage are replaced with the agent and the amount of damage,
        but [HEALTHINDICATOR] must be as it is in front. It should not be replaced in any way. The damage should not accumulate. Only output the new damage value and not the total damage.
        There may be other words such as hit or damaged or any other word like that to reference damage.
        Example responses include: '[HEALTHINDICATOR] Clove -120' or '[HEALTHINDICATOR] Astra -40'

        If a teammate calls out damage without an agent, then replace the agent with ALL


        The following are valid Valorant Agents:
        brimstone phoenix sage sova viper cypher reyna killjoy breach omen jett raze skye yoru astra kayo chamber neon fade harbor gekko deadlock iso clove
        There may be abbreviations used for each agent, such as KJ for killjoy or DL for deadlock.
""")

In [40]:
from flask import Flask, request, jsonify
from pyngrok import conf, ngrok
from google.colab import userdata
import threading
import json

app = Flask(__name__)
conf.get_default().auth_token = userdata.get('NGROKToken')


public_url = ngrok.connect("80", domain="cheaply-caring-pup.ngrok-free.app").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, "80"))

app.config["BASE_URL"] = public_url

# basic, will improve later
# may need to change this later if its not needed
class ValorantChat:
    def __init__(self, channel, user, line):
        self.channel = channel
        self.user = user
        self.line = line

    def Empty(self):
        self.channel = ""
        self.user = ""
        self.line = ""

    def json(self):
        return json.dumps(
            {
                "channel": self.channel,
                "user": self.user,
                "line": self.line
            }
        )

    def raw(self):
        return f"({self.channel}) {self.user}{self.line}"

@app.route('/', methods=['POST'])
def processChat():
    chatmsg = json.loads(request.json)

    chatmsg = ValorantChat(chatmsg["channel"], chatmsg["user"], chatmsg["line"])

    print(chatmsg.raw())
    response = llm.generateConversationResponse(chatmsg.line)
    print(response + "\n")
    return response
    pass

def start():
    app.run(host='0.0.0.0', port=80)

start()

 * ngrok tunnel "https://cheaply-caring-pup.ngrok-free.app" -> "http://127.0.0.1:80"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://172.28.0.12:80
INFO:werkzeug:Press CTRL+C to quit


(Party) iopi: sage -90


INFO:werkzeug:127.0.0.1 - - [11/Aug/2024 18:40:46] "POST / HTTP/1.1" 200 -


[HEALTHINDICATOR] Sage -90

(Party) iopi: reyna -100


INFO:werkzeug:127.0.0.1 - - [11/Aug/2024 18:40:57] "POST / HTTP/1.1" 200 -


[HEALTHINDICATOR] Reyna -100

